In [ ]:
import xarray as xr
import numpy as np
import dask
from dask.diagnostics import ProgressBar
import matplotlib.pylab as plt
import my_funs

from dask.distributed import Client, LocalCluster

# cluster = LocalCluster()
# client = Client(cluster)
# client = Client(processes=False)

dir = "/data/home/hamiddashti/hamid/nasa_above/greeness/"
# out_dir = "/data/home/hamiddashti/hamid/nasa_above/greeness/working/"

percent_cover = (
    xr.open_dataarray(dir +
                      "data/processed_data/percent_cover/percent_cover.nc") *
    100)
percent_cover = percent_cover.loc["1984":"2013"]
percent_cover = percent_cover.round(4)
# If a class is 0 change it no nan to prevent false zeros in diff later
percent_cover = percent_cover.where(percent_cover != 0)

Function to calculate natural variability


In [ ]:
def calulate_lcc(var, trend_ds, var_name, percent_cover, out_dir,thresh):

    var_trend = trend_ds["trend"]
    var_trend_total = var_trend.where(trend_ds["h"] == 1)  # where trend is sigificant

    # There is a tiny difference between lat and lon of the dataset due to rounding
    # We make them equal
    percent_cover["lat"] = var_trend_total["lat"]
    percent_cover["lon"] = var_trend_total["lon"]

    t_n = percent_cover.shape[0]
    band_n = percent_cover.shape[1]
    lat_n = percent_cover.shape[2]
    lon_n = percent_cover.shape[3]
    n_total = np.isfinite(var.isel(time=0)).sum()

    # Some setting
    winsize = 7  # Window size to search for stable pixels around a central pixel
    # thresh = 5  # Threshold to consider a change in ladn cover happended over the past 30 years

    [var_trend_nv, var_trend_lcc, var_trend_nv_lcc,
    included_pixels] = my_funs.estimate_lcc_trend(percent_cover, var_trend_total,
                                               thresh, winsize)
    # var_included = var.where(included_pixels)
    # var_trend_nv.to_netcdf(out_dir + var_name + "_trend_nv.nc")
    # var_trend_lcc.to_netcdf(out_dir + var_name + "_trend_lcc.nc")
    # var_trend_nv_lcc.to_netcdf(out_dir + var_name + "_trend_nv_lcc.nc")
    # var_included.to_netcdf(out_dir + var_name + "_included.nc")
    # included_pixels.to_netcdf(out_dir + "included_pixels.nc")
    return [var_trend_nv, var_trend_lcc, var_trend_nv_lcc]


Calculate natural variablity & LCC components of trend for LAI and NDVI annual max in both true values and normalized ones


In [ ]:
var_name = "lai_max"
out_dir = dir + "data/processed_data/noaa_nc/lai_fapar/trend/"
var = xr.open_dataarray(
    dir +
    "data/processed_data/noaa_nc/lai_fapar/resampled/lai_annual_resample_max.nc"
).sel(time=slice("1985", "2014")).rename({
    "latitude": "lat",
    "longitude": "lon"
})

trend_ds = xr.open_dataset(
    dir + "data/processed_data/noaa_nc/lai_fapar/trend/lai_max_trend.nc")


In [ ]:
calulate_lcc(var=var,
             trend_ds=trend_ds,
             var_name=var_name,
             percent_cover=percent_cover,
             out_dir=out_dir,
             thresh=5)


In [ ]:
var_name = "lai_max_norm"
out_dir = dir + "data/processed_data/noaa_nc/lai_fapar/trend/"

var = xr.open_dataarray(
    dir +
    "data/processed_data/noaa_nc/lai_fapar/resampled/lai_annual_resample_max_norm.nc"
).sel(time=slice("1985", "2014"))

trend_ds = xr.open_dataset(
    dir + "data/processed_data/noaa_nc/lai_fapar/trend/lai_max_norm_trend.nc")

calulate_lcc(var=var,
             trend_ds=trend_ds,
             var_name=var_name,
             percent_cover=percent_cover,
             out_dir=out_dir,
             thresh=5)


In [ ]:
var_name = "ndvi_max"
out_dir = dir + "data/processed_data/landsat/trend/"

var = xr.open_dataarray(
    dir + "data/processed_data/landsat/resampled/ndvi_annual_max.nc").rename({
        "latitude":
        "lat",
        "longitude":
        "lon"
    }).sel(time=slice("1985", "2014"))

trend_ds = xr.open_dataset(
    dir + "data/processed_data/landsat/trend/ndvi_annual_max_trend.nc")

calulate_lcc(var=var,
             trend_ds=trend_ds,
             var_name=var_name,
             percent_cover=percent_cover,
             out_dir=out_dir,
             thresh=5)

In [ ]:
var_name = "gs"
out_dir = dir + "data/processed_data/landsat/trend/"
var = xr.open_dataarray(
    dir + "data/processed_data/landsat/resampled/gs.nc").drop("band")
trend_ds = xr.open_dataset(
    dir + "data/processed_data/landsat/trend/gs_trend.nc").drop("band")

calulate_lcc(var=var,
             trend_ds=trend_ds,
             var_name=var_name,
             percent_cover=percent_cover,
             out_dir=out_dir,
             thresh=5)

In [ ]:
var_name = "gs_norm"
out_dir = dir + "data/processed_data/landsat/trend/"

var = xr.open_dataarray(
    dir + "data/processed_data/landsat/resampled/gs_norm.nc").drop("band")
trend_ds = xr.open_dataset(
    dir + "data/processed_data/landsat/trend/gs_trend_norm.nc").drop("band")

calulate_lcc(var=var,
             trend_ds=trend_ds,
             var_name=var_name,
             percent_cover=percent_cover,
             out_dir=out_dir,
             thresh=5)